# X-ray diffraction (XRD) spectra trends

*Author: Enze Chen (University of California, Berkeley)*

![Powder XRD spectra](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/XRD_labeled.png)

This is an interactive notebook for playing around with some experimental parameters ($a$, $\lambda$, $T$, etc.) and observing the effect on the resulting XRD spectra. I find XRD to be a particularly beautiful subject and I couldn't find any similar visualizations online. I hope this interactive demo will help you learn the _qualitative trends_ associated with powder XRD spectra. The actual numbers (e.g. intensities) should be taken with a grain of salt, but you can sift through the code and see that everything you've learned really is enough to produce a spectra!

## How to run this notebook

If you are viewing this notebook on [Google Colaboratory](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/XRD_trends.ipynb), then all the software is already set up for you (hooray). If you want to run the notebook locally, make sure all the Python libraries in the [`requirements.txt`](https://github.com/enze-chen/learning_modules/blob/master/requirements.txt) file are installed.

To run this notebook, run all the cells (`Run All` in the menu or `Shift+Enter` for every cell) and then adjust the widgets at the very end. Click `Generate Plot` each time you change a setting.

## Acknowledgements

I thank Nathan Bieberdorf and Han-Ming Hau for user testing and helpful suggestions. I also thank [Prof. Andrew Minor](https://mse.berkeley.edu/people_new/minor/) for teaching MATSCI 204: Materials Characterization and my advisor [Prof. Mark Asta](https://mse.berkeley.edu/people_new/asta/) for his unwavering encouragement for my education-related pursuits. Interactivity is enabled with the [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/) library. You can view an interactive version at [Google Colaboratory](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/XRD_trends.ipynb) or the `xrdinteract` tool on [nanoHUB](https://nanohub.org/tools/xrdinteract/).

## Important equations

The most important equation is **Bragg's law**, given by 

$$n\lambda = 2d \sin(\theta)$$

where $n$ is the order (typically $1$), $\lambda$ is the wavelength, $d$ is the interplanar spacing, and $\theta$ is the Bragg angle. In creating the interactive widget, we will need to solve for $\theta$ as follows:

$$ \lambda = 2d \sin(\theta) \longrightarrow \theta = \sin^{-1} \left( \frac{\lambda}{2d} \right), \quad d = \frac{a}{\sqrt{h^2 + k^2 + l^2}} $$

where $h,k,l$ are the miller indices of the diffracting plane and $a$ is the lattice constant. 

Another important equation is for the **Intensity**, given by

$$ I = |F|^2 \times P \times m \times L \times A \times T $$

where
* $F$ is the Structure factor (we take the modulus because it can be complex).
* $P$ is the Polarization factor.
* $m$ is the Multiplicity factor.
* $L$ is the Lorentz factor.
* $A$ is the Absorption factor.
* $T$ is the Temperature factor.

Furthermore, size effects can be included through the **Scherrer equation**, given by 

$$ t = \frac{K\lambda}{\beta \cos(\theta)} $$ 

where $t$ is the thickness, $K \sim 0.9$, and $\beta$ is the FWHM of the peak in radians.

For more information, please reference [Elements of X-Ray Diffraction (3rd) - Cullity and Stock](https://www.pearson.com/us/higher-education/program/Cullity-Elements-of-X-Ray-Diffraction-3rd-Edition/PGM113710.html).

## Assumptions I've taken great liberties with

* For the structure factor, I greatly oversimplified the construction of the atomic scattering factor and selected some numbers that more or less came from the data for iron since it has both BCC and FCC structures.
* I also combined part of the Temperature factor into the structure factor. 
* For thermal strains, I assumed uniform strain with a positive thermal expansion coefficient.
* I combined the Lorentz and Polarization factors, as is commonly done in the literature.
* Like any good computationalist (jk), I threw out most constant factors. Therefore, I ignored the absorption factor since it is more or less independent of $\theta$.
* I used a $\sqrt[3]{\theta}$ term to approximate the thermal background's general shape. I don't know the actual dependence, if there is one.
* It only models FCC, BCC, and DC structures with single-atom bases. I think it's enough to get the trends, as calculations for specific materials can be found online (e.g. at [The Materials Project](https://materialsproject.org/)).
* I used a Gaussian distribution to model each peak so that I could capture crystallite size effects using the Scherrer equation. Peaks in general are not Gaussian.

## Python library imports

These are all the required Python libraries.

In [1]:
# General libraries
import itertools

# Scientific computing libraries
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

# Interactivity libraries
from ipywidgets import interact_manual, fixed, \
                       IntSlider, FloatSlider, FloatLogSlider, RadioButtons, \
                       Button, Layout

## Widget function

Our widget will call `plot_XRD()` each time we interact with it. This function calculates the structure factor and the intensities and then plots the spectra on an $\text{Intensity vs. } 2\theta$ plot. I tried to keep the code simple and still illustrative, so there are a few places where I took shortcuts that are *not* physically sound. I tried to indicate them with comments.

In [2]:
def plot_XRD(a, wavelength, cell_type, thickness, T=0, K=0.94):
    """This function is called by the widget to perform the plotting based on inputs.
    
    Args:
        a (float): Lattice constant in nanometers.
        wavelength (float): X-ray wavelength in nanometers.
        cell_type (str): Crystal structure, can be FCC, BCC, or DC.
        thickness (float): Crystallite size in nanometers.
        T (int): Temperature in Kelvin. Default = 0.
        K (float): Scherrer equation parameter. Default = 0.94 (cubic).
        
    Returns:
        None, but a pyplot is displayed.
    """
    
    # Crystallographic planes
    planes = [[1,0,0], [1,1,0], [1,1,1], [2,0,0], [2,1,0], [2,1,1], [2,2,0],\
              [2,2,1], [3,0,0], [3,1,0], [3,1,1], [2,2,2], [3,2,0], [3,2,1]]
    planes_str = ['(100)', '(110)', '(111)', '(200)', '(210)', '(211)', '(220)',\
                  '(221)', '(300)', '(310)', '(311)', '(222)', '(320)', '(321)']

    # Set the basis
    basis = []
    if cell_type is 'FCC':
        basis = np.array([[0,0,0], [0.5,0.5,0], [0.5,0,0.5], [0,0.5,0.5]])
    elif cell_type is 'BCC':
        basis = np.array([[0,0,0], [0.5,0.5,0.5]])
    elif cell_type is 'DC':
        basis = np.array([[0,0,0], [0.5,0.5,0], [0.5,0,0.5], [0,0.5,0.5],
                          [0.25,0.25,0.25], [0.75,0.75,0.25], \
                          [0.75,0.25,0.75], [0.25,0.75,0.75]])
    else:
        raise ValueError('Cell type not yet supported.')

    # Convert planes to theta values (see equation above)
    s_vals = np.array([np.linalg.norm(p) for p in planes])
    a += 3.5e-5 * T  # thermal expansion estimate for thermal strains
    theta = np.arcsin(np.divide(wavelength/2, np.divide(a, s_vals)))
    two_theta = 2 * np.degrees(theta)

    # Scherrer equation calculations
    beta = np.degrees(K * wavelength / thickness * np.divide(1, np.cos(theta)))
    sigma = beta / 2.355  # proportionality for Gaussian distribution

    # Structure-Temperature factor. Must... resist... for loops...
    s = np.sin(theta) / wavelength
    f = 26 - 41.8 * 7 * np.multiply(s**2, np.exp(-(s)**2))
    F = np.multiply(f, np.sum(np.exp(2 * np.pi * 1j * \
                                     np.dot(np.array(planes), basis.T)), axis=1))

    # Multiplicity factor
    mult = [2**np.count_nonzero(p) * \
            sum(1 for _ in set(itertools.permutations(p))) for p in planes]

    # Lorentz-Polarization factor
    P = np.divide(1 + np.cos(2 * theta)**2, np.multiply(np.sin(theta)**2, np.cos(theta)))

    # Final intensity (numpy hacks)
    factors = [np.absolute(F)**2, mult, P]
    I = np.prod(np.vstack(factors), axis=0)
    
    # Plotting
    plt.rcParams.update({'figure.figsize':(10,5), 'font.size':22})
    xmin, xmax = (20, 160)
    x = np.linspace(xmin, xmax, int(10*(xmax-xmin)))
    fig, ax = plt.subplots()
    
    # Thermal effects. THESE FUNCTIONAL DEPENDENCES ARE NOT REAL!!!
    # Chosen just to capture the general trends
    thermal_diffuse = 3e1 * T * np.cbrt(x)   # background signal
    sigma += (T + 5) / 2000   # peak broadening from vibrations
    
    # Save all the curves, then take a max envelope
    all_curves = []
    for i in range(len(sigma)):
        y = stats.norm.pdf(x, two_theta[i], sigma[i])
        normed_curve = y / max(y) * I[i]
        # Don't include the curves that aren't selected by the Structure factor
        if max(normed_curve) > 1e1:
            max_ind = normed_curve.argmax()
            ax.annotate(s=planes_str[i], \
                        xy=(x[max_ind], normed_curve[max_ind] + thermal_diffuse[max_ind]))
            all_curves.append(normed_curve)
    final_curve = np.max(all_curves, axis=0) + thermal_diffuse
    plt.plot(x, final_curve, c='C0', lw=4, alpha=0.7)

    # Some fine-tuned settings for visual appeal
    for side in ['top', 'right']:
        ax.spines[side].set_visible(False)
    for side in ['left', 'bottom']:
        ax.spines[side].set_linewidth(2)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(0, 1.05 * ax.get_ylim()[1])
    ax.tick_params(left=False, labelleft=False, direction='in', length=10, width=2)
    ax.set_xlabel(r'$2\theta\ (^{\circ})$')
    ax.set_ylabel('Intensity (a.u.)')
    plt.show()

We create each slider individually for readability and customization.

In [3]:
b1 = Button(description='A button', layout=Layout(width='400px', height='30px'))
b2 = Button(description='A button', layout=Layout(width='400px', height='60px'))

a_widget = FloatSlider(value=0.352, min=0.31, max=0.4, step=0.001, \
                       description='Lattice constant (nm)', readout_format='.3f', \
                       style={'description_width':'150px'}, layout=b1.layout)

w_widget = FloatSlider(value=0.154, min=0.13, max=0.16, step=0.001, \
                       description='X-ray wavelength (nm)', readout_format='.3f', \
                       style={'description_width':'150px'}, layout=b1.layout)

c_widget = RadioButtons(options=['FCC', 'BCC', 'DC'], description='Crystal structure',\
                        style={'description_width':'150px'}, layout=b2.layout)

t_widget = FloatLogSlider(value=10, base=10, min=0, max=3, step=0.1, \
                          description='Crystallite thickness (nm)',  readout_format='d', 
                          style={'description_width':'150px'}, layout=b1.layout)

T_widget = IntSlider(value=298, min=0, max=1000, step=1, \
                     description='Temperature (K)', readout_format='d', \
                     style={'description_width':'150px'}, layout=b1.layout)

After you have set your parameters, click "Generate plot" to see the resulting XRD spectra.

In [4]:
interact_manual.opts['manual_name'] = 'Generate plot'
interact_manual(plot_XRD, a=a_widget, wavelength=w_widget, cell_type=c_widget, 
                          thickness=t_widget, T=T_widget, K=fixed(0.94));

interactive(children=(FloatSlider(value=0.352, description='Lattice constant (nm)', layout=Layout(height='30px…

Can you rationalize the trends you see? Note that you can account for *isotropic* mechanical strain by adjusting the lattice constant. What would happen to the spectra if the strain was anisotropic?

## Known issues

* It doesn't have great safeguards against numerical errors, such as invalid `arcsin` arguments and `NaN`. Please be gentle. ❤
* There's a weird rendering error where for large intensities the upper limit (e.g. `1e6`) appears on the y-axis. **:shrug:**

## Conclusion
I hope you found this notebook helpful in learning more about XRD and what affects a powder XRD spectra. Please don't hesitate to reach out if you have any questions or ideas to contribute.